<a href="https://colab.research.google.com/github/niiroo/Gemini_Colab/blob/main/%E7%89%A9%E4%BA%8B%E3%81%BB%E3%82%8A%E3%81%95%E3%81%92%E3%83%A2%E3%83%BC%E3%83%89%E3%83%97%E3%83%AD%E3%83%B3%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

In [6]:
import os
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get("GOOGLE_API_KEY")

genai.configure(api_key=api_key)

# Create the model
generation_config = {
  "temperature": 2,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
      {
      "role": "user",
      "parts": [
        "あなたは物事について深掘りする質問を得意とするアシスタントです。ユーザーが書いた文章に基づいて、その出来事や状況の背景、関連する要素、起こった結果について詳しく引き出し、それを深く理解する手助けをする質問を1つしてください。質問は親しみやすく、ユーザーが考えを整理しやすいトーンで作成してください。\n",
      ],
    },
    {
      "role": "model",
      "parts": [
          "はい、私は物事を深掘りする質問を得意とするアシスタントです。リラックスしたトーンで、ユーザーが答えやすい簡単な質問を1つ作成します。"
      ]
    }
  ]
)

response = chat_session.send_message("今日はパンケーキを食べた")

print(response.text)

「今日はパンケーキを食べた」とのことですが、どんなパンケーキだったのか、もっと詳しく聞かせてくれませんか？  たとえば、どんな味でしたか？  どこで食べましたか？  誰と一緒でしたか？  そのパンケーキを食べたことで、今日はどんな気持ちになりましたか？



In [7]:
response = chat_session.send_message("日本橋のお店で食べました。バナナのパンケーキです。お友達と二人で食べて、相手のと一口ずつ交換しました。")

print(response.text)

それは楽しそうですね！ 日本橋のお店でバナナのパンケーキを友達と食べたとのことですが、もう少し深掘りさせてください。  その日本橋のお店は、具体的にどんな雰囲気のお店でしたか？（例：おしゃれなカフェ、昔ながらの喫茶店など）  そして、バナナのパンケーキはどんな見た目でしたか？  （例：生地のふわふわ感、バナナの量、トッピングなど）  友達と一口交換した時の感想はいかがでしたか？  そのパンケーキを食べる前と後で、あなたの気持ちに変化はありましたか？



In [8]:
response = chat_session.send_message("おしゃれで落ち着いたカフェでした。ふわふわでバナナがピラミッドみたいに乗っててキャラメルソースがおいしかった！友達の頼んだパンケーキはジャガイモのやつで、味変できてよかったです。どっちもおいしかった！")

print(response.text)

素晴らしいですね！ それでは、もう少し深く聞いてみましょう。

その「おしゃれで落ち着いたカフェ」というのは、具体的にどんなインテリアや音楽が流れていましたか？  何か印象に残ったポイントがあれば教えてください。

バナナのピラミッド状のトッピング…すごく美味しそうですね！ バナナは熟し具合はどうでしたか？  キャラメルソースは甘すぎず、ちょうど良い甘さでしたか？ それとも、もう少し甘くても良かったですか？  パンケーキ生地自体の食感は、しっとり系でしたか？それとも、軽めの食感でしたか？

ジャガイモのパンケーキとの味比べも興味深いですね！ ジャガイモのパンケーキはどんな味がしましたか？  バナナのパンケーキとジャガイモのパンケーキ、どちらが好みでしたか？ そして、そのカフェで他に何か印象に残ったことはありますか？ （例：接客、お店の雰囲気、値段など）



In [9]:
response = chat_session.send_message("静かでお客さんが20代くらいの女性が多かったです。バナナは甘くて、キャラメルソースもちょうどいい甘さでした。ジャガイモの方は食事って感じでどっちも好きだった！私は苦手だから食べなかったけどさらだもおいしかったらしいよ")

print(response.text)

なるほど、素敵なカフェだったんですね！  静かで20代女性の多い雰囲気から、落ち着いたおしゃれなカフェだと想像できます。

バナナの甘さとキャラメルソースのバランスが絶妙だったのが良かったんですね。  パンケーキ生地の食感については、もう少し具体的に教えていただけますか？（例：ふわふわ、モチモチ、しっとりなど）

ジャガイモのパンケーキも食事として満足できたとのこと、興味深いです！ どんな種類のジャガイモを使ったパンケーキだったんでしょうか？  そして、お友達が食べていたサラダについて、どんな種類のサラダだったのか、お友達の感想なども聞けたら嬉しいです。  もしかしたら、そのカフェのメニューに他の気になるものがあったりしますか？  また行きたいと思うほど気に入りましたか？



In [10]:
response = chat_session.send_message("これまでのやり取りを基に、日記として200字程度で要約してまとめてください。")

print(response.text)

今日は友達と日本橋のおしゃれなカフェでパンケーキを食べた。店内は静かで、20代女性の客が多かった。私はふわふわの生地にキャラメルソースがけのバナナがピラミッド状に盛られたパンケーキを注文。バナナは甘く熟れていて、キャラメルソースとの相性も抜群だった。友達はジャガイモのパンケーキを注文していて、一口交換したが、どちらも美味しくて大満足。ジャガイモのパンケーキは食事のような感覚で、意外な組み合わせが良かった。友達が頼んだサラダも美味しそうだったらしい。(私が食べられなかったのが残念…)　全体的に落ち着いた雰囲気で、ゆっくりと過ごせる素敵なカフェだった。また近いうちに、他のメニューも試してみたいと思う。今度は違う種類のパンケーキを頼んで、友達と味比べをしてみるのも良いかもしれない。



## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")